In [52]:
import pandas as pd
import aaindexer



In [53]:
data = pd.read_excel("H:\\Documents\\VUB\\advanced_methods_in_bioInformatics\\BioInformatics\\data_structerd.xlsx")
data = data.sample(n=1000, random_state=42)


In [54]:
print(data.head(5))
type(data["Mutation"])
#type(data["Mutation"][4])


       Unnamed: 0 UniProt ID   Gene Mutation      Class
7308         7308     P12110  CO6A2    R366Q     Driver
635           635     Q96NR3  PTHD1    I139T     Driver
16581       16581     Q15813   TBCE    V158L     Driver
16705       16705     P01266   THYG   R1423G  Passenger
4978         4978     Q01484   ANK2   E2109K  Passenger


pandas.core.series.Series

## Physicochemical properties

In [55]:
n= 10 # number of features to extract fro aindex data base 
aindex_base = aaindexer.scrape_parse(1, progress=False)[:n]

In [56]:
aindex_base[0]

AaindexRecord(accession='ANDN920101', description='alpha-CH chemical shifts (Andersen et al., 1992)', pmid='PMID:1575719', authors='Andersen, N.H., Cao, B. and Chen, C.', title='Peptide/protein structure analysis using the chemical shift index method:  upfield alpha-CH values reveal dynamic helices and aL sites', journal='Biochem. and Biophys. Res. Comm. 184, 1008-1014 (1992)', comment=None, correlation={'BUNA790102': 0.949}, index={'A': 4.35, 'R': 4.38, 'N': 4.75, 'D': 4.76, 'C': 4.65, 'Q': 4.37, 'E': 4.29, 'G': 3.97, 'H': 4.63, 'I': 3.95, 'L': 4.17, 'K': 4.36, 'M': 4.52, 'F': 4.66, 'P': 4.44, 'S': 4.5, 'T': 4.35, 'W': 4.7, 'Y': 4.6, 'V': 3.95}, matrix=None)

In [57]:
for a_row in aindex_base:
    prop = a_row.accession

    # Initialize the new column if not already present
    if prop not in data.columns:
        data[prop] = None

    for index, d_row in data.iterrows():
        try:
            wild_residue = d_row["Mutation"][0]
            mutant_residue = d_row["Mutation"][-1]

            # Ensure both residues are in the index dictionary
            if mutant_residue in a_row.index and wild_residue in a_row.index:
                diff = a_row.index[mutant_residue] - a_row.index[wild_residue]
                data.at[index, prop] = diff
            else:
                data.at[index, prop] = None  # or np.nan
        except Exception as e:
            print(f"Error at row {index}: {e}")
            data.at[index, prop] = None

        
    

        
    

In [58]:
print(data.head(10))


       Unnamed: 0 UniProt ID   Gene Mutation      Class ANDN920101 ARGP820101  \
7308         7308     P12110  CO6A2    R366Q     Driver      -0.01       -0.6   
635           635     Q96NR3  PTHD1    I139T     Driver        0.4      -2.17   
16581       16581     Q15813   TBCE    V158L     Driver       0.22       0.21   
16705       16705     P01266   THYG   R1423G  Passenger      -0.41      -0.53   
4978         4978     Q01484   ANK2   E2109K  Passenger       0.07       0.68   
10174       10174     Q96RW7  HMCN1   R5205L  Passenger      -0.21       0.93   
7882         7882     P11532    DMD   C1892Y  Passenger      -0.05       0.81   
9725         9725     Q6P531   GGT6     G52R     Driver       0.41       0.53   
4100         4100     O95177  GAAS1     V60I     Driver        0.0        0.9   
4201         4201     Q5VV41  ARHGG    A552T     Driver        0.0      -0.56   

      ARGP820102 ARGP820103 BEGF750101 BEGF750102 BEGF750103 BHAR880101  \
7308        0.52       0.06      

## Di- and tri-peptide motifs:

In [59]:
pip install biopython pandas requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
import pandas as pd
import requests
from Bio import SeqIO
from io import StringIO

In [61]:
data["bf_peptide"]= None 
data["af_peptide"]= None
data["bf_peptide_ratio"]= None
data["af_peptide_ratio"]= None

In [62]:
import pandas as pd
import re
import requests
from collections import defaultdict

# Step 1: Fetch UniProt sequences
def fetch_uniprot_sequence(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    try:
        r = requests.get(url)
        if r.ok:
            lines = r.text.strip().split('\n')
            return ''.join(lines[1:])  # remove FASTA header
    except Exception as e:
        print(f"Error fetching {uniprot_id}: {e}")
    return None

# Assume 'data' is your DataFrame containing 'UniProt ID', 'Mutation', 'Class' columns
uniprot_ids = data['UniProt ID'].unique()
uniprot_sequences = {uid: fetch_uniprot_sequence(uid) for uid in uniprot_ids}

def extract_dipeptides(seq, mutation, uid=None):
    match = re.match(r"([A-Z])(\d+)([A-Z])", mutation)
    if not match:
        return None, None
    wt_aa, pos_str, mut_aa = match.groups()
    pos = int(pos_str) - 1  # Convert 1-based to 0-based index

    # Check valid position for dipeptides extraction
    if pos <= 0 or pos >= len(seq) - 1:
        return None, None

    # Verify wild-type AA matches sequence at mutation position
    if seq[pos] != wt_aa:
        print(f"⚠️ Warning: Wild-type mismatch at position {pos+1} for {uid}. Expected {wt_aa}, found {seq[pos]}.")
        return None, None

    before = seq[pos - 1] + seq[pos]
    after = seq[pos] + seq[pos + 1]

    return before, after

# Step 3: Count dipeptides by class
dipeptide_counts = defaultdict(lambda: {'Driver': 0, 'Passenger': 0})

for _, row in data.iterrows():
    uid = row['UniProt ID']
    seq = uniprot_sequences.get(uid)
    if not seq:
        continue
    before, after = extract_dipeptides(seq, row['Mutation'], uid=uid)
    data.at[row.name, 'bf_peptide'] = before
    data.at[row.name, 'af_peptide'] = after
    for dipep in [before, after]:
        if dipep:
            dipeptide_counts[dipep][row['Class']] += 1

# Step 4: Compute odds ratios
results = []

total_driver = sum(c['Driver'] for c in dipeptide_counts.values())
total_passenger = sum(c['Passenger'] for c in dipeptide_counts.values())

if total_driver == 0 or total_passenger == 0:
    print("⚠️ Cannot compute odds ratios: one of the classes has zero total.")
else:
    for dipep, counts in dipeptide_counts.items():
        ndp = counts['Driver']
        nnp = counts['Passenger']
        odds = (ndp / total_driver) / (nnp / total_passenger) if nnp > 0 else float('inf')

        if odds >= 1.2:
            label = "i"  # highly_preferred
        elif odds <= 0.8:
            label = "ii"  # not_preferred
        else:
            label = "c"  # Moderate

        results.append((dipep, ndp, nnp, round(odds, 2), label))

# Step 5: Display results
odds_df = pd.DataFrame(results, columns=["Dipeptide", "Driver_count", "Passenger_count", "Odds_ratio", "Class"])
print(odds_df)


⚠️ Warning: Wild-type mismatch at position 60 for O95177. Expected V, found I.
⚠️ Warning: Wild-type mismatch at position 3868 for Q4G0P3. Expected M, found T.
⚠️ Warning: Wild-type mismatch at position 1366 for Q9HCJ0. Expected R, found L.
⚠️ Warning: Wild-type mismatch at position 12024 for Q8WZ42. Expected L, found G.
⚠️ Warning: Wild-type mismatch at position 186 for P26640. Expected A, found N.
⚠️ Warning: Wild-type mismatch at position 4411 for Q15149. Expected A, found T.
⚠️ Warning: Wild-type mismatch at position 417 for B1ANS9. Expected R, found Y.
⚠️ Warning: Wild-type mismatch at position 173 for O95843. Expected G, found F.
⚠️ Warning: Wild-type mismatch at position 9761 for Q8WZ42. Expected P, found I.
⚠️ Warning: Wild-type mismatch at position 46 for Q9UPP2. Expected R, found L.
⚠️ Warning: Wild-type mismatch at position 218 for Q8NH37. Expected Y, found V.
⚠️ Warning: Wild-type mismatch at position 516 for P45844. Expected R, found A.
⚠️ Warning: Wild-type mismatch at po

In [63]:

for index, row in data.iterrows():
    bf_peptide = row['bf_peptide']
    af_peptide = row['af_peptide']
    
    if bf_peptide : 
        ratio_class= odds_df[odds_df["Dipeptide"] == bf_peptide]["Class"]
        if not ratio_class.empty:
            data.at[index, "bf_peptide_ratio"] = ratio_class.values[0]
    if af_peptide:
        ratio_class= odds_df[odds_df["Dipeptide"] == af_peptide]["Class"]
        if not ratio_class.empty:
            data.at[index, "af_peptide_ratio"] = ratio_class.values[0]

In [64]:

for index, row in data.iterrows():
    bf_peptide = row['bf_peptide']
    af_peptide = row['af_peptide']

    if bf_peptide:
        ratio_class = odds_df[odds_df["Dipeptide"] == bf_peptide]["Odds_ratio"]
        if not ratio_class.empty:
            data.at[index, "bf_peptide_ratio"] = ratio_class.values[0]
    if af_peptide:
        ratio_class = odds_df[odds_df["Dipeptide"] == af_peptide]["Odds_ratio"]
        if not ratio_class.empty:
            data.at[index, "af_peptide_ratio"] = ratio_class.values[0]

In [65]:
data.head(10)

,Unnamed: 0,UniProt ID,Gene,Mutation,Class,ANDN920101,ARGP820101,ARGP820102,ARGP820103,BEGF750101,BEGF750102,BEGF750103,BHAR880101,BIGC670101,BIOV880101,bf_peptide,af_peptide,bf_peptide_ratio,af_peptide_ratio
7308,7308,P12110,CO6A2,R366Q,Driver,-0.01,-0.6,0.52,0.06,-0.08,0.0,-0.2,-0.036,-19.4,-3.0,ER,RG,1.22,2.9
635,635,Q96NR3,PTHD1,I139T,Driver,0.4,-2.17,-0.61,-0.76,-0.29,-0.15,0.38,-0.018,-30.8,-189.0,HI,IC,1.09,inf
16581,16581,Q15813,TBCE,V158L,Driver,0.22,0.21,2.15,1.79,0.18,-0.15,0.16,-0.021,16.9,22.0,KV,VD,1.09,1.09
16705,16705,P01266,THYG,R1423G,Passenger,-0.41,-0.53,0.29,0.17,-0.17,-0.07,0.13,0.015,-72.8,57.0,IR,RF,5.98,2.18
4978,4978,Q01484,ANK2,E2109K,Passenger,0.07,0.68,-0.05,-0.08,-0.13,0.0,0.22,-0.031,20.4,-35.0,SE,EV,0.31,1.63
10174,10174,Q96RW7,HMCN1,R5205L,Passenger,-0.21,0.93,3.03,2.48,0.48,0.11,-0.31,-0.164,-7.1,215.0,QR,RC,0.87,0.31
7882,7882,P11532,DMD,C1892Y,Passenger,-0.05,0.81,-1.5,-0.55,0.38,0.18,0.0,0.074,47.9,-115.0,KC,CL,0.54,0.0
9725,9725,Q6P531,GGT6,G52R,Driver,0.41,0.53,-0.29,-0.17,0.17,0.07,-0.13,-0.015,72.8,-57.0,GG,GL,1.63,3.26
4100,4100,O95177,GAAS1,V60I,Driver,0.0,0.9,0.37,0.53,-0.09,0.0,0.0,0.076,16.9,28.0,None,None,None,None
4201,4201,Q5VV41,ARHGG,A552T,Driver,0.0,-0.56,-0.34,-0.65,-0.56,0.06,0.38,0.087,18.6,-54.0,YA,AQ,2.18,0.44


In [66]:
print(data[data["Class"] == "Driver"].shape)
print(data.shape)


(515, 19)
(1000, 19)


Identifying the deptides that are highly preferred:

In [67]:
passenger_preferred_before_dipeptides = data[data['bf_peptide_ratio'] == 0.0]
print("Passenger preferred dipeptides (Odds ratio = 0.0):")
print(passenger_preferred_before_dipeptides[['Mutation', 'bf_peptide', 'bf_peptide_ratio', 'af_peptide', 'af_peptide_ratio']])
# Save the results to csv:
passenger_preferred_before_dipeptides.to_csv("passenger_preferred_before_dipeptides.csv", index=False)

Passenger preferred dipeptides (Odds ratio = 0.0):
      Mutation bf_peptide bf_peptide_ratio af_peptide af_peptide_ratio
6051    S1914N         CS              0.0         ST             0.73
6427     I666V         SI              0.0         IM              0.0
12650   T1899I         HT              0.0         TY              0.0
7887    N2355K         YN              0.0         NQ             0.22
13484    H246N         IH              0.0         HA             0.36
14092    H923R         IH              0.0         HI             1.09
12696    I109T         TI              0.0         ID             0.73
10141     C70Q         WC              0.0         CL              0.0
12273    P513L         SP              0.0         PC              0.0
10652    D580H         MD              0.0         DS             0.33
11188    S813N         CS              0.0         SG             0.87
11998    P831A         SP              0.0         PL             1.09
15547   W1095C         TW 

In [68]:
passenger_preferred_after_dipeptides = data[data['af_peptide_ratio'] == 0.0]
print("Passenger preferred dipeptides (Odds ratio = 0.0):")
print(passenger_preferred_after_dipeptides[['Mutation', 'af_peptide', 'af_peptide_ratio']])
# Save the results to csv:
passenger_preferred_after_dipeptides.to_csv("passenger_preferred_after_dipeptides.csv", index=False)

Passenger preferred dipeptides (Odds ratio = 0.0):
      Mutation af_peptide af_peptide_ratio
7882    C1892Y         CL              0.0
15303   N1715D         NI              0.0
13517    T476I         TY              0.0
6427     I666V         IM              0.0
14241    A295T         AN              0.0
12650   T1899I         TY              0.0
6351     C847Y         CG              0.0
10141     C70Q         CL              0.0
12273    P513L         PC              0.0
8928    K2009E         KS              0.0
15547   W1095C         WS              0.0
15511    D349N         DH              0.0
8509     S379C         SY              0.0
12523    G341V         GC              0.0
11977   G1316A         GH              0.0
7517     E249Q         EC              0.0
8341     A206T         AW              0.0
9809    T5650I         TI              0.0
12370    I578V         IH              0.0
17641   H2185R         HD              0.0
6682     N748S         NH              0.0
120

In [69]:
highly_preferred_after_dipeptides = data[data['af_peptide_ratio'] == float('inf')]
print("Highly preferred dipeptides (Odds ratio = ∞):")
print(highly_preferred_after_dipeptides[['Mutation', 'af_peptide', 'af_peptide_ratio']])
# Save the results to csv:
highly_preferred_after_dipeptides.to_csv("highly_preferred_dipeptides.csv", index=False)

Highly preferred dipeptides (Odds ratio = ∞):
      Mutation af_peptide af_peptide_ratio
635      I139T         IC              inf
15566   M1424I         ML              inf
7592      N16Y         NY              inf
5642     C604Y         CE              inf
5702     H332Y         HQ              inf
1020      R69C         RY              inf
828      L185M         LW              inf
13058    H823N         HG              inf
856      I164M         IK              inf
1624      D37H         DC              inf
15947    N775S         NY              inf
3268     C120F         CD              inf
4413     F208L         FG              inf
103      G121V         GD              inf
2167     G366D         GN              inf
16615   R1745C         RY              inf
4660     G680D         GF              inf
1166     G274V         GW              inf
1944    H1095R         HC              inf
1261     L323R         LF              inf
16860   D1069N         DC              inf
10739   

In [ ]:
highly_preferred_before_dipeptides = data[data['bf_peptide_ratio'] == float('inf')]
print("Highly preferred dipeptides (Odds ratio = ∞):")
print(highly_preferred_before_dipeptides[['Mutation', 'bf_peptide', 'bf_peptide_ratio']])
# Save the results to csv:
highly_preferred_before_dipeptides.to_csv("highly_preferred_before_dipeptides.csv", index=False)

Highly preferred dipeptides (Odds ratio = ∞):
      Mutation bf_peptide bf_peptide_ratio
1554      G41R         FG              inf
10889    M112K         MM              inf
11072    Q229E         GQ              inf
7592      N16Y         DN              inf
2127     G222E         HG              inf
13163   G1025S         FG              inf
16562    R146K         HR              inf
13058    H823N         PH              inf
10025    R467W         WR              inf
1398     R844H         WR              inf
4413     F208L         KF              inf
1944    H1095R         YH              inf
5149      M58V         HM              inf
11225    W181C         LW              inf
15178   N1017K         EN              inf
9473     P143S         IP              inf
1355     M332V         FM              inf
736      K103N         FK              inf
3669       L5F         WL              inf
14057   M2804T         CM              inf
4153     G171R         HG              inf
2325    

In [ ]:
highly_preferred_dipeptides = data[(data['bf_peptide_ratio'] == float('inf')) & (data['af_peptide_ratio'] == float('inf'))]
print("Highly preferred dipeptides (Odds ratio = ∞):")
print(highly_preferred_dipeptides[['Mutation', 'bf_peptide', 'bf_peptide_ratio', 'af_peptide', 'af_peptide_ratio']])
# Save the results to csv:
highly_preferred_dipeptides.to_csv("highly_preferred_dipeptides.csv", index=False)


Highly preferred dipeptides (Odds ratio = ∞):
      Mutation bf_peptide bf_peptide_ratio af_peptide af_peptide_ratio
7592      N16Y         DN              inf         NY              inf
13058    H823N         PH              inf         HG              inf
4413     F208L         KF              inf         FG              inf
1944    H1095R         YH              inf         HC              inf
5149      M58V         HM              inf         ML              inf
11225    W181C         LW              inf         WL              inf
3669       L5F         WL              inf         LW              inf
4153     G171R         HG              inf         GD              inf
9547      W14C         DW              inf         WL              inf


In [76]:
# List of dipeptides to look for (both before and after peptides)
target_dipeptides = [
    "DN", "NY",
    "PH", "HG",
    "KF", "FG",
    "YH", "HC",
    "HM", "ML",
    "LW", "WL",
    "WL", "LW",
    "HG", "GD",
    "DW", "WL"
]

# Remove duplicates in case any exist
target_dipeptides = list(set(target_dipeptides))

# Filter odds_df for these dipeptides
filtered_df = odds_df[odds_df['Dipeptide'].isin(target_dipeptides)][['Dipeptide', 'Driver_count', 'Passenger_count']]

print(filtered_df)


    Dipeptide  Driver_count  Passenger_count
66         FG             3                0
105        ML             2                0
163        DN             2                0
164        NY             2                0
178        HG             3                0
217        LW             3                0
223        PH             1                0
264        KF             1                0
265        GD             4                0
285        YH             1                0
286        HC             1                0
299        HM             1                0
314        WL             3                0
346        DW             1                0


In [78]:
passenger_preferred_dipeptides = data[(data['bf_peptide_ratio'] == 0) & (data['af_peptide_ratio'] == 0)]
print("Passenger preferred dipeptides (Odds ratio = 0):")
print(passenger_preferred_dipeptides[['Mutation', 'bf_peptide', 'bf_peptide_ratio', 'af_peptide', 'af_peptide_ratio']])
# Save the results to csv:
passenger_preferred_dipeptides.to_csv("passenger_preferred_dipeptides.csv", index=False)

Passenger preferred dipeptides (Odds ratio = 0):
      Mutation bf_peptide bf_peptide_ratio af_peptide af_peptide_ratio
6427     I666V         SI              0.0         IM              0.0
12650   T1899I         HT              0.0         TY              0.0
10141     C70Q         WC              0.0         CL              0.0
12273    P513L         SP              0.0         PC              0.0
15547   W1095C         TW              0.0         WS              0.0
9809    T5650I         HT              0.0         TI              0.0
17641   H2185R         TH              0.0         HD              0.0
8451     M739L         TM              0.0         MR              0.0
11153   T2702A         NT              0.0         TI              0.0
6053     S231N         KS              0.0         SY              0.0
17471    T668M         HT              0.0         TH              0.0
5589     I254L         QI              0.0         IN              0.0
12078     Y15H         SY   

In [70]:
data = data.drop(columns=["Unnamed: 0"])

In [71]:
data.to_csv("features_data_1.csv", index=False)

In [72]:
# have the final _data

data_2= pd.read_csv("faraah_my_final_merged_featuress.csv")

final_data = pd.merge(data, data_2, on=["Mutation","UniProt ID" ], how="inner")
final_data.dropna(inplace=True)
final_data.to_csv("final_data.csv", index=False)
final_data.shape

(923, 251)